In [1]:
# CELL 1: Install Dependencies
# ============================================================
# This cell installs the required Python package for making HTTP requests

import subprocess
subprocess.run(['pip', 'install', 'requests', '-q'], check=True)
print("✓ Dependencies installed successfully")


✓ Dependencies installed successfully


In [3]:
# ============================================================
# Import necessary modules for API calls and datetime handling

import requests
import json
from datetime import datetime, timedelta, timezone

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


In [4]:
# CELL 3: Configure Your API Credentials
# ============================================================
# Replace these with your actual credentials from Vapi Dashboard

VAPI_API_KEY = "620a9d44-150f-4ded-9030-b77d6174abab"  # Get from: https://dashboard.vapi.ai
VAPI_BASE_URL = "https://api.vapi.ai"

# Your Vapi resources
ASSISTANT_ID = "f551a9ca-ee53-4b65-bf9c-49c0c13bba7b"           # Your saved assistant ID from Vapi Dashboard
PHONE_NUMBER_ID = "4205d53a-86ab-42dd-aab7-4cd54bfcb612"     # Your imported or Vapi number
CUSTOMER_PHONE = "+33939036481"              # Replace with actual number to call

print("Configuration loaded:")
print(f"  API Base URL: {VAPI_BASE_URL}")
print(f"  Assistant ID: {ASSISTANT_ID[:20]}..." if len(ASSISTANT_ID) > 20 else f"  Assistant ID: {ASSISTANT_ID}")
print(f"  Phone Number ID: {PHONE_NUMBER_ID[:20]}..." if len(PHONE_NUMBER_ID) > 20 else f"  Phone Number ID: {PHONE_NUMBER_ID}")
print(f"  Customer Phone: {CUSTOMER_PHONE}")

Configuration loaded:
  API Base URL: https://api.vapi.ai
  Assistant ID: f551a9ca-ee53-4b65-b...
  Phone Number ID: 4205d53a-86ab-42dd-a...
  Customer Phone: +33939036481


In [5]:
# CELL 4: Create Transient Assistant Configuration
# ============================================================
# A transient assistant is a temporary assistant defined inline
# Use this if you don't have a saved assistant in Vapi Dashboard

transient_assistant = {
    "name": "French Support Assistant",
    "model": {
        "provider": "openai",
        "model": "gpt-3.5-turbo",
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful customer service assistant. You speak French and English. Keep responses concise and professional."
            }
        ]
    },
    "voice": {
        "provider": "openai",
        "voiceId": "alloy"  # Options: alloy, echo, fable, onyx, nova, shimmer
    }
}

print("✓ Transient assistant configuration created")
print(json.dumps(transient_assistant, indent=2))



✓ Transient assistant configuration created
{
  "name": "French Support Assistant",
  "model": {
    "provider": "openai",
    "model": "gpt-3.5-turbo",
    "messages": [
      {
        "role": "system",
        "content": "You are a helpful customer service assistant. You speak French and English. Keep responses concise and professional."
      }
    ]
  },
  "voice": {
    "provider": "openai",
    "voiceId": "alloy"
  }
}


In [6]:
# CELL 5: Helper Function - Make Single Immediate Call
# ============================================================
# This function makes an outbound call immediately to one phone number

def make_single_call(customer_number, assistant_id=None, transient_assistant=None):
    """
    Make an immediate outbound call to a single phone number.

    Args:
        customer_number (str): Phone number in E.164 format (e.g., "+33123456789")
        assistant_id (str): ID of a saved assistant in Vapi
        transient_assistant (dict): Inline assistant configuration

    Returns:
        tuple: (response_dict, status_code)
    """

    # Validate inputs
    if not assistant_id and not transient_assistant:
        raise ValueError("Must provide either assistant_id or transient_assistant")

    # Set up request headers
    headers = {
        "Authorization": f"Bearer {VAPI_API_KEY}",
        "Content-Type": "application/json"
    }

    # Build the request payload
    payload = {
        "phoneNumberId": PHONE_NUMBER_ID,
        "customer": {
            "number": customer_number
        }
    }

    # Use either saved assistant or transient
    if assistant_id:
        payload["assistantId"] = assistant_id
    else:
        payload["assistant"] = transient_assistant

    # Make the API request
    response = requests.post(
        f"{VAPI_BASE_URL}/call",
        headers=headers,
        json=payload
    )

    return response.json(), response.status_code


print("✓ Single call function defined")

✓ Single call function defined


In [7]:
# CELL 6: Helper Function - Schedule a Call for Later
# ============================================================
# This function schedules a call to be placed at a specific time in the future

def schedule_single_call(customer_number, earliest_time, latest_time=None,
                        assistant_id=None, transient_assistant=None):
    """
    Schedule an outbound call for a specific future time.

    Args:
        customer_number (str): Phone number in E.164 format
        earliest_time (str): ISO 8601 datetime string (e.g., "2025-11-16T10:00:00Z")
        latest_time (str, optional): Latest time to attempt the call
        assistant_id (str): ID of a saved assistant
        transient_assistant (dict): Inline assistant configuration

    Returns:
        tuple: (response_dict, status_code)
    """

    if not assistant_id and not transient_assistant:
        raise ValueError("Must provide either assistant_id or transient_assistant")

    headers = {
        "Authorization": f"Bearer {VAPI_API_KEY}",
        "Content-Type": "application/json"
    }

    # Create schedule plan with time window
    schedule_plan = {
        "earliestAt": earliest_time
    }
    if latest_time:
        schedule_plan["latestAt"] = latest_time

    payload = {
        "phoneNumberId": PHONE_NUMBER_ID,
        "customer": {
            "number": customer_number
        },
        "schedulePlan": schedule_plan
    }

    if assistant_id:
        payload["assistantId"] = assistant_id
    else:
        payload["assistant"] = transient_assistant

    response = requests.post(
        f"{VAPI_BASE_URL}/call",
        headers=headers,
        json=payload
    )

    return response.json(), response.status_code


print("✓ Schedule call function defined")



✓ Schedule call function defined


In [8]:
# CELL 7: Helper Function - Make Batch Calls
# ============================================================
# This function makes multiple outbound calls at once

def make_batch_calls(customer_numbers, assistant_id=None, transient_assistant=None):
    """
    Make outbound calls to multiple phone numbers simultaneously.

    Args:
        customer_numbers (list): List of phone numbers in E.164 format
        assistant_id (str): ID of a saved assistant
        transient_assistant (dict): Inline assistant configuration

    Returns:
        tuple: (response_dict, status_code)
    """

    if not assistant_id and not transient_assistant:
        raise ValueError("Must provide either assistant_id or transient_assistant")

    headers = {
        "Authorization": f"Bearer {VAPI_API_KEY}",
        "Content-Type": "application/json"
    }

    # Convert list of numbers to customer objects
    customers = [{"number": num} for num in customer_numbers]

    payload = {
        "phoneNumberId": PHONE_NUMBER_ID,
        "customers": customers
    }

    if assistant_id:
        payload["assistantId"] = assistant_id
    else:
        payload["assistant"] = transient_assistant

    response = requests.post(
        f"{VAPI_BASE_URL}/call",
        headers=headers,
        json=payload
    )

    return response.json(), response.status_code


print("✓ Batch calls function defined")

✓ Batch calls function defined


In [9]:
# CELL 8: Helper Function - Schedule Batch Calls
# ============================================================
# This function schedules multiple calls for a specific time

def schedule_batch_calls(customer_numbers, earliest_time, latest_time=None,
                        assistant_id=None, transient_assistant=None):
    """
    Schedule outbound calls to multiple numbers for a specific time.

    Args:
        customer_numbers (list): List of phone numbers in E.164 format
        earliest_time (str): ISO 8601 datetime string
        latest_time (str, optional): Latest time to attempt calls
        assistant_id (str): ID of a saved assistant
        transient_assistant (dict): Inline assistant configuration

    Returns:
        tuple: (response_dict, status_code)
    """

    if not assistant_id and not transient_assistant:
        raise ValueError("Must provide either assistant_id or transient_assistant")

    headers = {
        "Authorization": f"Bearer {VAPI_API_KEY}",
        "Content-Type": "application/json"
    }

    customers = [{"number": num} for num in customer_numbers]

    schedule_plan = {
        "earliestAt": earliest_time
    }
    if latest_time:
        schedule_plan["latestAt"] = latest_time

    payload = {
        "phoneNumberId": PHONE_NUMBER_ID,
        "customers": customers,
        "schedulePlan": schedule_plan
    }

    if assistant_id:
        payload["assistantId"] = assistant_id
    else:
        payload["assistant"] = transient_assistant

    response = requests.post(
        f"{VAPI_BASE_URL}/call",
        headers=headers,
        json=payload
    )

    return response.json(), response.status_code


print("✓ Schedule batch calls function defined")


✓ Schedule batch calls function defined


In [10]:
# CELL 9: Helper Function - Pretty Print Results
# ============================================================
# This function nicely formats and displays API responses

def print_result(title, result, status_code):
    """
    Print API call results in a readable format.

    Args:
        title (str): Title for the result
        result (dict): Response dictionary from API
        status_code (int): HTTP status code
    """
    print(f"\n{'='*60}")
    print(f"{title}")
    print(f"{'='*60}")
    print(f"Status Code: {status_code}")

    if status_code == 201:
        print("✓ SUCCESS")
        if "id" in result:
            print(f"Call ID: {result['id']}")
        if "status" in result:
            print(f"Call Status: {result['status']}")
        if "customer" in result:
            print(f"Customer Number: {result['customer'].get('number')}")
        if "schedulePlan" in result:
            print(f"Scheduled At: {result['schedulePlan'].get('earliestAt')}")
    else:
        print("✗ ERROR")
        if "message" in result:
            print(f"Error Message: {result['message']}")

    print(f"\nFull Response:")
    print(json.dumps(result, indent=2))


print("✓ Print helper function defined")


✓ Print helper function defined


In [11]:
# CELL 10: Example 1 - Make a Single Immediate Call
# ============================================================
# This example makes an outbound call immediately to one phone number

print("\n" + "="*60)
print("EXAMPLE 1: Single Immediate Call")
print("="*60)
print("Description: Make an immediate outbound call to one phone number")
print("Phone Number: " + CUSTOMER_PHONE)

try:
    result, status = make_single_call(
        customer_number=CUSTOMER_PHONE,
        assistant_id=ASSISTANT_ID  # Using your saved assistant
    )
    print_result("Single Immediate Call Result", result, status)

except Exception as e:
    print(f"✗ Error: {e}")


EXAMPLE 1: Single Immediate Call
Description: Make an immediate outbound call to one phone number
Phone Number: +33939036481

Single Immediate Call Result
Status Code: 201
✓ SUCCESS
Call ID: 019a8a98-78f1-788c-8b92-300828e7cab9
Call Status: queued
Customer Number: +33939036481

Full Response:
{
  "subscriptionLimits": {
    "concurrencyBlocked": false,
    "concurrencyLimit": 10,
    "remainingConcurrentCalls": 9
  },
  "id": "019a8a98-78f1-788c-8b92-300828e7cab9",
  "assistantId": "f551a9ca-ee53-4b65-bf9c-49c0c13bba7b",
  "phoneNumberId": "4205d53a-86ab-42dd-aab7-4cd54bfcb612",
  "type": "outboundPhoneCall",
  "createdAt": "2025-11-16T02:57:19.601Z",
  "updatedAt": "2025-11-16T02:57:21.438Z",
  "orgId": "16f1304c-41f0-4a0f-bd06-313d1c60c0d9",
  "cost": 0,
  "customer": {
    "number": "+33939036481"
  },
  "status": "queued",
  "phoneCallProvider": "twilio",
  "phoneCallProviderId": "CA59d15d7df47d4d87c97d0451ad2029bc",
  "phoneCallTransport": "pstn",
  "monitor": {
    "listenUrl": 

In [12]:
# CELL 11: Example 2 - Schedule a Call for Later
# ============================================================
# This example schedules a call to be placed 1 hour from now

print("\n" + "="*60)
print("EXAMPLE 2: Schedule a Call for 1 Hour from Now")
print("="*60)
print("Description: Schedule an outbound call for a future time")

try:
    # Calculate future time (1 hour from now)
    future_time = (datetime.now(timezone.utc) + timedelta(hours=1)).isoformat().replace('+00:00', 'Z')
    print(f"Scheduled Time: {future_time}")

    result, status = schedule_single_call(
        customer_number=CUSTOMER_PHONE,
        earliest_time=future_time,
        assistant_id=ASSISTANT_ID
    )
    print_result("Scheduled Call Result", result, status)

except Exception as e:
    print(f"✗ Error: {e}")




EXAMPLE 2: Schedule a Call for 1 Hour from Now
Description: Schedule an outbound call for a future time
Scheduled Time: 2025-11-16T03:57:28.745007Z

Scheduled Call Result
Status Code: 201
✓ SUCCESS
Call ID: 019a8a98-9d89-7aa9-a068-fbb0dbb6ee77
Call Status: scheduled
Customer Number: +33939036481
Scheduled At: 2025-11-16T03:57:28.745Z

Full Response:
{
  "subscriptionLimits": {
    "concurrencyBlocked": false,
    "concurrencyLimit": 10,
    "remainingConcurrentCalls": 7
  },
  "id": "019a8a98-9d89-7aa9-a068-fbb0dbb6ee77",
  "assistantId": "f551a9ca-ee53-4b65-bf9c-49c0c13bba7b",
  "phoneNumberId": "4205d53a-86ab-42dd-aab7-4cd54bfcb612",
  "type": "outboundPhoneCall",
  "createdAt": "2025-11-16T02:57:28.969Z",
  "updatedAt": "2025-11-16T02:57:28.969Z",
  "orgId": "16f1304c-41f0-4a0f-bd06-313d1c60c0d9",
  "cost": 0,
  "customer": {
    "number": "+33939036481"
  },
  "status": "scheduled",
  "schedulePlan": {
    "earliestAt": "2025-11-16T03:57:28.745Z"
  }
}


In [13]:
# ============================================================
# CELL 12: Example 3 - Make Batch Calls (Multiple Numbers)
# ============================================================
# This example makes calls to multiple phone numbers at once

print("\n" + "="*60)
print("EXAMPLE 3: Batch Calls to Multiple Numbers")
print("="*60)
print("Description: Make immediate calls to multiple phone numbers")

# List of phone numbers to call (replace with real numbers)
phone_numbers = [
    "+33123456789",  # Replace with real French number
    "+33987654321",  # Replace with real French number
    "+33555123456"   # Replace with real French number
]

print(f"Numbers to Call: {phone_numbers}")

try:
    result, status = make_batch_calls(
        customer_numbers=phone_numbers,
        assistant_id=ASSISTANT_ID
    )
    print_result("Batch Calls Result", result, status)

except Exception as e:
    print(f"✗ Error: {e}")


EXAMPLE 3: Batch Calls to Multiple Numbers
Description: Make immediate calls to multiple phone numbers
Numbers to Call: ['+33123456789', '+33987654321', '+33555123456']

Batch Calls Result
Status Code: 201
✓ SUCCESS

Full Response:
{
  "results": [
    {
      "id": "019a8a98-a436-7cc8-93e8-97901c837cd2",
      "assistantId": "f551a9ca-ee53-4b65-bf9c-49c0c13bba7b",
      "phoneNumberId": "4205d53a-86ab-42dd-aab7-4cd54bfcb612",
      "type": "outboundPhoneCall",
      "createdAt": "2025-11-16T02:57:30.678Z",
      "updatedAt": "2025-11-16T02:57:31.742Z",
      "orgId": "16f1304c-41f0-4a0f-bd06-313d1c60c0d9",
      "cost": 0,
      "customer": {
        "number": "+33123456789"
      },
      "status": "queued",
      "phoneCallProvider": "twilio",
      "phoneCallProviderId": "CAc6c3fe1f1b1b10f959022b86d8a74c15",
      "phoneCallTransport": "pstn",
      "monitor": {
        "listenUrl": "wss://phone-call-websocket.oci-us-sanjose-1-backend-production1.vapi.ai/019a8a98-a436-7cc8-93e8-97

In [14]:
# ============================================================
# CELL 13: Example 4 - Schedule Batch Calls
# ============================================================
# This example schedules calls to multiple numbers for a specific time

print("\n" + "="*60)
print("EXAMPLE 4: Schedule Batch Calls for 2 Hours from Now")
print("="*60)
print("Description: Schedule calls to multiple numbers for a future time")

phone_numbers = [
    "+33123456789",  # Replace with real French number
    "+33987654321"   # Replace with real French number
]

try:
    # Calculate future time (2 hours from now)
    future_time = (datetime.now(timezone.utc) + timedelta(hours=2)).isoformat().replace('+00:00', 'Z')
    print(f"Scheduled Time: {future_time}")
    print(f"Numbers to Call: {phone_numbers}")

    result, status = schedule_batch_calls(
        customer_numbers=phone_numbers,
        earliest_time=future_time,
        assistant_id=ASSISTANT_ID
    )
    print_result("Scheduled Batch Calls Result", result, status)

except Exception as e:
    print(f"✗ Error: {e}")



EXAMPLE 4: Schedule Batch Calls for 2 Hours from Now
Description: Schedule calls to multiple numbers for a future time
Scheduled Time: 2025-11-16T04:57:34.380126Z
Numbers to Call: ['+33123456789', '+33987654321']

Scheduled Batch Calls Result
Status Code: 201
✓ SUCCESS

Full Response:
{
  "results": [
    {
      "id": "019a8a98-b43e-755d-b61b-0291267b4550",
      "assistantId": "f551a9ca-ee53-4b65-bf9c-49c0c13bba7b",
      "phoneNumberId": "4205d53a-86ab-42dd-aab7-4cd54bfcb612",
      "type": "outboundPhoneCall",
      "createdAt": "2025-11-16T02:57:34.782Z",
      "updatedAt": "2025-11-16T02:57:34.782Z",
      "orgId": "16f1304c-41f0-4a0f-bd06-313d1c60c0d9",
      "cost": 0,
      "customer": {
        "number": "+33123456789"
      },
      "status": "scheduled",
      "schedulePlan": {
        "earliestAt": "2025-11-16T04:57:34.380Z"
      }
    },
    {
      "id": "019a8a98-b98b-755d-b61b-0fe75cee259e",
      "assistantId": "f551a9ca-ee53-4b65-bf9c-49c0c13bba7b",
      "phoneNum